In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv")
mydata = data
mydata.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# FOR FUZZY MATCH
import fuzzywuzzy
from fuzzywuzzy import process, fuzz

# FOR TRANSLATE URDU-ENGLISH
from textblob import TextBlob

In [ ]:
mydata = mydata.rename({"Order Number": "OrdNumber", 
                        "Order Status": "OrdStatus", 
                        "Book Name" : "BookName", 
                        "Order Date & Time": "OrdDate", 
                        "City": "city", 
                        "Payment Method": "PayMethod", 
                        "Total items": "TotItems", 
                        "Total weight (grams)": "weight"}, 
                       axis = 1 )
mydata.head()

In [ ]:
mydata.city =mydata.city.astype('str')
mydata.city = mydata.city.str.lower()
mydata.city = mydata.city.str.strip()
provided_data = mydata.city.value_counts()
provided_data

In [ ]:
print (mydata.city.value_counts().sum())
print ("Unique Cities: ", len(mydata.city.unique()))

In [ ]:
short_names = mydata.city.str.len()<=4
print ( mydata[short_names].city.unique())
print ("\nUnique Cities whith Short Name: ", len(mydata[short_names].city.unique()) )

In [ ]:
short_to_full = {
    "fsd": "faisalabad",
    "rwp": "rawalpindi",
    "lhr": "lahore",
    "isb": "islamabad",
    "ryk": "rahim yar khan",    
    "khi": "karachi",
    "hyd": "hydrabad",
    "aj&k": "azad jammu & kashmir",
    "lahr": "lahore",
    "lah": "lahore",
    "d i khan": "dera ismail khan",
    "g g khan": "dera ghazi khan",
    "dgk": "dera ghazi khan",
    "vehari": "vihari",
    "mailsi, district vehari": "vihari"
}
mydata.city = mydata.city.replace(short_to_full)

In [ ]:
after_short_names = mydata.city.str.len()<=4
mydata[after_short_names].city.unique()
print ( mydata[after_short_names].city.unique())
print ("\nUnique Cities whith Short Name: ", len(mydata[after_short_names].city.unique()) )

In [ ]:
alpha_num_name = mydata.city.str.contains("[a-zA-Z0-9]")
beore_urduNames = mydata[~alpha_num_name].city.unique()
beore_urduNames.shape

In [ ]:
len(beore_urduNames), beore_urduNames

In [ ]:
convert_to_eng = {}
for x in beore_urduNames:
    try:
        tr = TextBlob(x).translate().string
        convert_to_eng[x] = tr
    except:
        pass

In [ ]:
convert_to_eng

In [ ]:
mydata.city = mydata.city.replace(convert_to_eng)

In [ ]:
alpha_num_name = mydata.city.str.contains("[a-zA-Z0-9]")
after_urduNames = mydata[~alpha_num_name].city.unique()
after_urduNames.shape

In [ ]:
after_urduNames

In [ ]:
remaing_translate_to_eng = {
    "خانیوال": "khanewal",
    "ہیلاں": "heelan",
    "اوکاڑہ": "okara",
    "پھالیہ": "phalia",
    "پیرمحل": "pir mahal",
    "لیہ": "layyah",
    "سنانواں": "sananwan",
    "جعفراباد": "jaffarabad",
    "کوہیٹا": "kohita",
    "روہیلانوالی": "rohilanwali",
    "ہرنائی": "harnai",
    "ہنگو": "hangu",
    "محلہ گوڑھا": "mohalla godha",
    "ٹمن": "timon",
    "ہرنولی": "harnoli",
    "سکہر": "sukkur",
    "جام پور": "jampur",
    "ميرپورخاص": "mirpurkhas",
    "لالیاں": "lalian",
    "بھکر": "bhakkar",
    "پلندری": "pallandri",
    "کنڈیارو": "kandiaro",
    "دھوک مصاحب": "dhok musahib",
    "صالح پٹ": "saleh Pat",
    "لودھراں": "lodhran",
    "پتوکی": "patoki",
    "چوک آعظم": "chowk azam"
}
mydata.city = mydata.city.replace(remaing_translate_to_eng)

In [ ]:
alpha_num_name = mydata.city.str.contains("[a-zA-Z0-9]")
after_replace_urduNames = mydata[~alpha_num_name].city.unique()
after_replace_urduNames.shape

In [ ]:
mydata.city.str.lower()
mydata.city.value_counts()

Use External Data for Cities Normalization

In [ ]:
externalData = pd.read_csv("../input/pakistan-cities/pk.csv")
externalData.info()

In [ ]:
externalData.city = externalData.city.str.lower()
externalData.city = externalData.city.str.strip()
externalData.head()

In [ ]:
externalData[externalData.city.str.contains('city')]

In [ ]:
externalData.city = externalData.city.replace('sialkot city', 'sialkot')
externalData.city = externalData.city.replace('jhang city', 'jhang')
externalData.city = externalData.city.replace('hyderabad city', 'hyderabad')
externalData.city = externalData.city.replace('attock city', 'attock')

In [ ]:
testExt = {}
mydata.new_city = None
mydata.lat = None
mydata.lng = None
mydata.ratio = None

for x, xRows in externalData.iterrows():
    for y, yRows in mydata.iterrows():
        if (xRows.city not in yRows.city):
            if xRows.city not in testExt:
                testExt[xRows.city] = {"count":1}
            else:
                testExt[xRows.city]["count"] = testExt[xRows.city]["count"] + 1
        else:
            getRatio = fuzz.ratio(yRows.city, xRows.city)
            mydata.at[y, 'ratio'] = getRatio
            mydata.at[y, 'new_city'] = xRows.city
            mydata.at[y, 'lat'] = xRows.lat
            mydata.at[y, 'lng'] = xRows.lng

In [ ]:
mydata.info()
mydata

In [ ]:
print ('-------- BEFORE CITIES --------\n', provided_data )

after_data = mydata['new_city'].value_counts()
print ('\n\n-------- AFTER CITIES --------\n', after_data)
#after_data = mydata.city.value_counts()

In [ ]:
print ("\nTotal Number of Rows: ", len(mydata))
print ("\n\n----------------- BEFORE PROCESSING --------------------\n")
print ("Unique Cities: ", len(mydata['city'].unique()))
print (f"Total Unique Cities: {mydata['city'].nunique()} " )
print ("\n\n------------------ AFTER PROCESSING --------------------\n")
print ("Unique Cities: ", len(mydata['new_city'].unique()))
print (f"Total Unique Cities: {mydata['new_city'].nunique()} " )
print ("\n\n---------- UNPROCCESSED/INVALID CITIES --------------\n")
print ("Number of Invalid / UnProccessed Cities: ", mydata['new_city'].isnull().sum())

Hope attempt is appriciated, looking for your guidance **like, UPVOTES and with lots of love**

**Only one confussion about textblob**: with same line of code why sometime not execute (uncertainty behaviour without any change in code), loop working fine but textblob statement skip from code and unable to get required record.
